In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('KaggleCovidDataset.csv').applymap(lambda x: 1 if x == "Yes" else 0)
df

/tmp/ipykernel_19206/2713265967.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = pd.read_csv('KaggleCovidDataset.csv').applymap(lambda x: 1 if x == "Yes" else 0)


,Breathing Problem,Fever,Dry Cough,Sore throat,Running Nose,Asthma,Chronic Lung Disease,Headache,Heart Disease,Diabetes,...,Fatigue,Gastrointestinal,Abroad travel,Contact with COVID Patient,Attended Large Gathering,Visited Public Exposed Places,Family working in Public Exposed Places,Wearing Masks,Sanitization from Market,COVID-19
0,1,1,1,1,1,0,0,0,0,1,...,1,1,0,1,0,1,1,0,0,1
1,1,1,1,1,0,1,1,1,0,0,...,1,0,0,0,1,1,0,0,0,1
2,1,1,1,1,1,1,1,1,0,1,...,1,1,1,0,0,0,0,0,0,1
3,1,1,1,0,0,1,0,0,1,1,...,0,0,1,0,1,1,0,0,0,1
4,1,1,1,1,1,0,1,1,1,1,...,0,1,0,1,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5429,1,1,0,1,1,1,1,0,0,0,...,1,1,0,0,0,0,0,0,0,1
5430,1,1,1,0,1,1,0,1,0,1,...,1,0,0,0,0,0,0,0,0,1
5431,1,1,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5432,1,1,1,0,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [45]:
def apriori_algorithm(df, min_support):
    df_copy = df.copy(deep=True)

    frequent_one_itemsets = df_copy.apply(lambda x: x.value_counts(normalize=True).get(1, 0)).loc[lambda x: x >= min_support].to_dict()
    frequent_one_itemsets = {frozenset((itemset,)): support for itemset, support in frequent_one_itemsets.items()}

    def generate_frequent_k_itemsets(frequent_itemsets, k):
        frequent_k_itemsets = set()
        
        for itemset1, itemset2 in itertools.combinations(frequent_itemsets, 2):
            union_itemset = itemset1.union(itemset2)
            if len(union_itemset) == k:
                subsets = [frozenset(combination) for combination in itertools.combinations(union_itemset, k - 1)]
                if all([subset in frequent_itemsets for subset in subsets]):
                    frequent_k_itemsets.add(union_itemset)

        frequent_k_itemsets_support = {}
        for k_itemset in frequent_k_itemsets:
            support = df_copy[list(k_itemset)].all(axis=1).mean()
            if support >= min_support:
                frequent_k_itemsets_support[k_itemset] = support
        
        return frequent_k_itemsets_support

    all_frequent_itemsets = {**frequent_one_itemsets}
    frequent_k_itemsets = frequent_one_itemsets
    k = 2
    while len(frequent_k_itemsets) > 0:
        frequent_k_itemsets = generate_frequent_k_itemsets(set(frequent_k_itemsets.keys()), k)
        all_frequent_itemsets.update(frequent_k_itemsets)
        k += 1

    return pd.DataFrame([(support, itemset) for itemset, support in all_frequent_itemsets.items()], columns=['support', 'itemset'])

In [46]:
def association_rules(df, min_confidence):
    frequent_itemsets = {frozenset(itemset):support for itemset, support in zip(df['itemset'], df['support'])}
    association_rules_list = []

    for itemset in frequent_itemsets:
        if len(itemset) > 1:
            for i in range(1, len(itemset)):
                antecedent_itemsets = list(itertools.combinations(itemset, i))
                for antecedent in antecedent_itemsets:
                    antecedent = frozenset(antecedent)
                    consequent = itemset.difference(antecedent)

                    confidence = frequent_itemsets[itemset] / frequent_itemsets[antecedent]
                    if confidence >= min_confidence:
                        association_rules_list.append((antecedent, consequent, confidence))
    
    return pd.DataFrame(association_rules_list, columns=['antecedent', 'consequent', 'confidence'])

In [50]:
association_rules(apriori_algorithm(df, 0.2), 0.2)

,antecedent,consequent,confidence
0,(Fever),(Abroad travel),0.484437
1,(Abroad travel),(Fever),0.844553
2,(Attended Large Gathering),(Contact with COVID Patient),0.628287
3,(Contact with COVID Patient),(Attended Large Gathering),0.578503
4,(Gastrointestinal ),(Breathing Problem),0.628381
...,...,...,...
7729,"(Breathing Problem, Visited Public Exposed Pla...",(Sore throat),0.903084
7730,"(COVID-19, Visited Public Exposed Places, Feve...",(Dry Cough),0.869258
7731,"(Breathing Problem, Visited Public Exposed Pla...",(Fever),0.855950
7732,"(COVID-19, Fever, Dry Cough, Sore throat, Brea...",(Visited Public Exposed Places),0.605017
